In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!sudo apt-get install wget

!wget -r -nv -N -c -np https://physionet.org/files/sleep-accel/1.0.0/

!mkdir ./dataset

!mv ./physionet.org/files/sleep-accel/1.0.0/* dataset

!rm -r ./physionet.org/

!find ./dataset -name "*.html" -type f -delete

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Last-modified header missing -- time-stamps turned off.
2021-06-02 06:36:01 URL:https://physionet.org/files/sleep-accel/1.0.0/ [925] -> "physionet.org/files/sleep-accel/1.0.0/index.html" [1]
2021-06-02 06:36:01 URL:https://physionet.org/robots.txt [22/22] -> "physionet.org/robots.txt" [1]
Last-modified header missing -- time-stamps turned off.
2021-06-02 06:36:01 URL:https://physionet.org/files/sleep-accel/1.0.0/heart_rate/ [4162] -> "physionet.org/files/sleep-accel/1.0.0/heart_rate/index.html" [1]
Last-modified header missing -- time-stamps turned off.
2021-06-02 06:36:01 URL:https://physionet.org/files/sleep-accel/1.0.0/labels/ [4278]

Steps:

- Download and unzip the dataset
- Load the files
- Pre-process the loaded files (crop to keep the part of interest)
- Merge files from each user selecting a specific window time frame.
- Export the resulting file to `.csv`.
- Repeat the process for all the users.

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
from enum import Enum

class Error(Enum):
    match_number_users = "[Error]: number of users in list does not match"
    match_user_id = "[Error]: user id does not match between lists"
    match_length_arrays = "[Error]: the length of the lists does not match"
    match_index = "[Error]: indexes are mismatched"
    generic_error = "[Error]"

In [3]:
# Preparing paths
data_path = "C:\dev\DATA\MRH"
# data_path = os.path.join(os.getcwd(), "dataset/")

motion_path = os.path.join(data_path, "motion")

heart_rate_path = os.path.join(data_path, "heart_rate")

labels_path = os.path.join(data_path, "labels")

# Obtaining ordered lists with all users
motion_list = sorted(os.listdir(motion_path))
heart_rate_list = sorted(os.listdir(heart_rate_path))
labels_list = sorted(os.listdir(labels_path))

# Checking that we have data of the 31 users in all the lists created
assert len(motion_list) == 31, Error.match_number_users.value
assert len(heart_rate_list) == 31, Error.match_number_users.value
assert len(labels_list) == 31, Error.match_number_users.value

# Checking that the user ids match in order accross the three lists
for item in range(len(motion_list)):
    user_motion_id = re.search("\d*", motion_list[item])
    user_heart_rate_id = re.search("\d*", heart_rate_list[item])
    user_labels_id = re.search("\d*", labels_list[item])

    assert user_motion_id.group(0) == user_heart_rate_id.group(0), Error.match_user_id.value
    assert user_motion_id.group(0) == user_labels_id.group(0), Error.match_user_id.value


In [4]:
motion_list[0], heart_rate_list[0], labels_list[0]

('1066528_acceleration.txt',
 '1066528_heartrate.txt',
 '1066528_labeled_sleep.txt')

In [21]:
user_1_motion = np.loadtxt(os.path.join(motion_path, motion_list[0]))

user_1_motion

array([[-2.16848465e+04,  7.08010000e-03,  6.40900000e-04,
        -9.87594600e-01],
       [-2.16848171e+04,  4.15040000e-03,  6.25600000e-04,
        -9.90554800e-01],
       [-2.16848079e+04,  4.15040000e-03,  1.11390000e-03,
        -9.90081800e-01],
       ...,
       [ 2.86265419e+04, -5.52734400e-01, -2.99988000e-02,
        -8.10440100e-01],
       [ 2.86265428e+04, -5.53710900e-01, -3.05023000e-02,
        -8.11431900e-01],
       [ 2.86265436e+04, -5.54718000e-01, -2.99988000e-02,
        -8.09021000e-01]])

In [22]:
user_1_heart_rate = np.loadtxt(os.path.join(heart_rate_path, heart_rate_list[0]), delimiter=',')

user_1_heart_rate

array([[-3.55241740e+05,  8.60000000e+01],
       [-3.51407999e+05,  6.70000000e+01],
       [-3.51277368e+05,  1.41000000e+02],
       ...,
       [ 2.91101643e+04,  7.50000000e+01],
       [ 3.43346538e+04,  8.10000000e+01],
       [ 3.44911535e+04,  6.50000000e+01]])

In [23]:
user_1_labels = np.loadtxt(os.path.join(labels_path, labels_list[0]))

user_1_labels

array([[    0.,     0.],
       [   30.,     0.],
       [   60.,     0.],
       ...,
       [28470.,     0.],
       [28500.,     0.],
       [28530.,     0.]])

In [7]:
def generate_dataset(motion_user, heart_rate_user, labels_user, interval_peak=1, interval_epoch=30):

    '''
    It accepts three filenames from one user to generate the dataset. Interval stands for the time in seconds of windowing.
    '''
    
    # --- Loading the txt files
    motion = np.loadtxt(motion_user)
    heart_rate = np.loadtxt(heart_rate_user, delimiter=',')
    labels = np.loadtxt(labels_user)

    
    # --- Cropping to match the labelled list
    motion = crop_to_offset(motion, labels)    
    heart_rate = crop_to_offset(heart_rate, labels)
    
    # --- Pre-processing and merging
    motion = get_summary_count(motion, interval_peak, interval_epoch)
    
    merged_arrays = merge_arrays(motion, heart_rate, labels)
    
  
  
    
    # OLD:
    # Extending smaller arrays to have the same size as the biggest array so as to be merged
    # It returns one dimensional array (time column skipped since it has been matched in the extending process)
    # heart_rate = extend_array(heart_rate, motion)
    # labels = extend_array(labels, motion)
    
    # Merging three arrays into one data frame
    # data_frame = pd.DataFrame(motion, columns=['Time', 'X', 'Y', 'Z'])
    
    # heart_rate_column = pd.Series(heart_rate)
    # data_frame["Heart Rate"] = heart_rate_column
    
    # labels_column = pd.Series(labels)
    # data_frame["Labels"] = labels_column
    
    # return data_frame
    return 1

The raw data recorded with the Apple Watch (motion and heart rate) contains continiuous and uninterrumped measurements of one or more days, including the last night.

Since the data corresponding to the last night underwent a proper labelling from the PSG results, it is necessary to crop the raw data only to that night (i.e. the list with labels). Anything else, will not be part of the generated dataset and will therefore be disregarded.

This is handled by the function `crop_to_offset()`. This function carries out two tasks:

1. It finds the last night measured within the array passed.
2. For the last night, it finds the boundaries corresponding to the start and end of the labelled list.

Then, the function returns the indexes where the array needs to be sliced.

In [35]:
def crop_to_offset(array_to_crop, array_ref):
    '''
    This function takes two arrays, the first is the one to be cropped and the second one the reference to where to crop.
    It returs a new array starting and ending where the indexes matched with the reference array.
    '''

    start_index, end_index = 0, 0
    array_size = np.size(array_to_crop, 0)
    cropped_array = []
    
    # --- Find the boundaries corresponding to the labelled list
    first_item = array_ref[0][0]
    last_item = array_ref[-1][0]
    
    last_item_found = False
    
    for item in range(array_size - 1, -1, -1):        
        # find end index
        if not last_item_found:
            if array_to_crop[item][0] < last_item:
                end_index = item + 1
                last_item_found = True
        
        # find start index
        if array_to_crop[item][0] < first_item:
            start_index = item
            break  # No more iteration is needed after finding end_index and start_index.
    
    
    # return (start_index, end_index)
    cropped_array = array_to_crop[start_index:end_index]
        
    return cropped_array

In order to compress the vast amount of data gathered from the IMU sensor, some pre-processing is required. Tipically, these types of sensors record at a high frequencies resulting in hundreds of measurements every single second. Research suggest that changes among sleep cycles tend to occur gradually within a few minutes. This also applies to the shift between NREM and REM, which is the variable of interest that is the focus of this application.

Therefore, the time resolution in the raw dataset is too accurate and the function `get_summary_count()` will handle this. All this function does is first finds the peak values within a user-defined time interval (we are most interested in peak values from the accelerometer that contribute to more valuable information) and sum all the spikes that take place within a window or time (epoch), also defined by the user.

An example might be that the function finds the spike that occur in every second and then sums all the spikes found within a window of 15 seconds. This process then is repeated until completion.

In [27]:
def get_summary_count(array, peak_interval, sum_window):
    '''
    This function first finds the peak value for every peak_interval (s) of the passed array.
    It then sums all the peak values within sum_window (s) and adds the sum to a new array.
    It returns the resulting processed array back.
    '''

    array_size = np.size(array, 0)
    peak_values_x, sum_peak_values_x = [], []
    peak_values_y, sum_peak_values_y = [], []
    peak_values_z, sum_peak_values_z = [], []
    
    # missing_indices = np.empty((0, 3), dtype=float)
    max_value_x, max_value_y, max_value_z = 0, 0, 0
    time_accumulate = []
    accumulate = 0
    last_interval = 0
    count = 0

    for item in range(array_size):
        if (array[item][0] - last_interval) < peak_interval:
            if abs(array[item][1]) > abs(max_value_x):  # New peak value found at x
                max_value_x = abs(array[item][1])

            if abs(array[item][2]) > abs(max_value_y):  # New peak value found at y
                max_value_y = abs(array[item][2])

            if abs(array[item][3]) > abs(max_value_z):  # New peak value found at z
                max_value_z = abs(array[item][3])

        # Gap found, do not continue with current window
        elif (array[item][0] - last_interval) > (peak_interval*1.5):
            # print("============================================")
            # print(f"[{array[item-1][0]}, {array[item][0]}]: {(array[item][0] - last_interval)}")
            # print("============================================")
                       
            # missing indices gets the time values of the gap between the missing data happened
            # new_interval = np.array([[array[item-1][0], array[item][0], (array[item][0] - array[item-1][0])]])
            # missing_indices = np.append(missing_indices, new_interval, axis=0)
            
            # update ,to the new time
            # print("acc: ", accumulate)
            accumulate = np.around(accumulate + (np.around(array[item][0], decimals=0) - array[item-1][0]) + count, decimals=0)
            # print("acc after: ", accumulate)
            # print("count: ", count)

            # reset for a new window time. Current unfinished window invalid.
            # if count < sum_window:              
            peak_values_x = []
            peak_values_y = []
            peak_values_z = []
            count = 0
            
            last_interval = np.floor(array[item][0])
            max_value_x = 0
            max_value_y = 0
            max_value_z = 0
            
        else:
            # end of peak interval
            peak_values_x.append(max_value_x)
            peak_values_y.append(max_value_y)
            peak_values_z.append(max_value_z)

            # reset interval values and increment count
            last_interval = np.floor(array[item][0])
            max_value_x = 0
            max_value_y = 0
            max_value_z = 0
            count += 1

        if count == sum_window:
            sum_peak_values_x.append(np.sum(peak_values_x))
            sum_peak_values_y.append(np.sum(peak_values_y))
            sum_peak_values_z.append(np.sum(peak_values_z))
            
            accumulate = np.around(accumulate + sum_window, decimals=0)

            # print("====== acc:", accumulate, "val:", np.around(array[item][0]), "diff:", (accumulate - array[item][0]))
            
            # Ensure that the current accumulated time matches the arrays' time
            assert abs(accumulate - np.around(array[item][0])) < 3, Error.match_index.value

            # time_accumulate.append(time_accumulate[-1] + sum_window)
            time_accumulate.append(accumulate)

            # reset for a new window time
            peak_values_x = []
            peak_values_y = []
            peak_values_z = []
            count = 0
       

    
    assert len(time_accumulate) == len(sum_peak_values_x), Error.match_length_arrays.value
    assert len(time_accumulate) == len(sum_peak_values_y), Error.match_length_arrays.value
    assert len(time_accumulate) == len(sum_peak_values_z), Error.match_length_arrays.value
    
    # return time_accumulate
    
    return np.column_stack((time_accumulate, sum_peak_values_x, sum_peak_values_y, sum_peak_values_z))

In [44]:
# user_1_motion = crop_to_offset(user_1_motion, user_1_labels)
# user_1_heart_rate = crop_to_offset(user_1_heart_rate, user_1_labels)

# fname = "cropped_" + motion_list[0]
# np.savetxt(fname, user_1_motion, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)

# from google.colab import files
# files.download('cropped_1066528_acceleration.txt') 

In [37]:
# [time.prev, time.next, diff]
summary = get_summary_count(user_1_motion, 1, 15)

summary

array([[1.50000000e+01, 6.12944040e+00, 6.76710490e+00, 1.20081636e+01],
       [3.00000000e+01, 6.11120610e+00, 6.92994680e+00, 1.19629060e+01],
       [4.50000000e+01, 6.09844980e+00, 6.90870630e+00, 1.19256594e+01],
       ...,
       [2.77820000e+04, 4.15765380e+00, 1.43766631e+01, 7.25236530e+00],
       [2.83730000e+04, 3.75987240e+00, 1.45188444e+01, 4.36595140e+00],
       [2.83880000e+04, 8.03883330e+00, 1.27173921e+01, 7.98275750e+00]])

Last step is merging the already pre-processed motion dataset with the heart rate and labels datasets corresponding to each user. 

For the heart rate dataset the following approach is taken. For each window from the motion time interval, gather all the heart rate values that were measured within that interval and get the mean value of all of them. If no heart rate value was measured within a window interval, the last known value will be taken. If there is a time gap bigger than the actual window's size, then discard that data up to the next available one.

For the labels a simpler approach is taken. Since the time windows are meant to be always smaller or equal than the time interval of labels recorded, the label value will be unique. It can be either one that corresponds to that time window or the last one if missing in that interval. 

All the above-mentioned operations are carried out in the `merge_arrays()` method.

In [6]:
def merge_arrays(motion, heart_rate, labels):
    array_size = max(np.size(motion, 0), np.size(heart_rate, 0), np.size(labels, 0))
    # Checking that the biggest arrray to iterate through at this point is heart_rate
    assert array_size == np.size(heart_rate, 0), Error.match_length_arrays

    heart_rate_mean = []
    heart_rate_acc = []
    labels_acc = []
    inc = 0

    for item in range(array_size):
        
        
        # Heart Rate pre-processing
        if heart_rate[item][0] < motion[inc][0]:
            heart_rate_acc.append(heart_rate[item][1])
        # elif heart_rate[item][0] - heart_rate[item - 1][0] > 15:
        #     heart_rate_mean.append()
        else:
            # print(f"[{motion[inc][0]}, {heart_rate[item][0]}]: ")
            # heart_rate_mean = heart_rate_acc/len(heart_rate_acc)
            # new_heart_rate.append(heart_rate_mean)
            heart_rate_mean.append(sum(heart_rate_acc)/len(heart_rate_acc))

            #if heart_rate[item][0] - motion[inc][0] > 15:
            #    heart_rate_mean.append(heart_rate[item][1])

            heart_rate_acc = [heart_rate[item][1]] # include first item that gave the condition too
            inc += 1
        
        # Labels pre-processing
        
        
        
    
    # Removing exceding data of motion that is not available in heart rate.
    if len(heart_rate_mean) < np.size(motion, 0):
        diff = abs(len(heart_rate_mean) - np.size(motion, 0)) #3
        last_row = np.size(motion, 0)
        start = last_row - diff

        subarray = np.arange(start, last_row)

        motion = np.delete(motion, subarray, axis=0)
    

    print(len(heart_rate_mean))
    print(np.size(motion, 0))

    
    return heart_rate_mean

In [112]:
arr = merge_arrays(summary, user_1_heart_rate, user_1_labels)

# np.size(summary, 0), len(arr)

3
1559
1559


In [96]:
arr1 = np.array([(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12), (13, 14, 15), (16, 17, 18), (50, 51, 52)])

diff = 3
last_row = np.size(arr1, 0)
print(last_row)
start = last_row - diff
print(start)

a = np.arange(start, last_row)

np.delete(arr1, a, axis=0)

7
4


array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [66]:
SIZE = np.size(user_1_heart_rate, 0)
for i in range(SIZE):

    if i ==  SIZE - 1:
        break
    if user_1_heart_rate[i+1][0] - user_1_heart_rate[i][0] > 15:
        print(user_1_heart_rate[i][0], user_1_heart_rate[i+1][0])

3386.1731801 3401.20835996
5712.87800002 5731.87800002
9734.31342006 9799.92673993
11673.9735799 11738.8798599
13322.02246 13341.8583701
15168.7177401 15195.5771501
17260.17623 17277.1761999
22938.1772201 22954.1772201
26092.6721699 26134.6679599
26134.6679599 26399.6651599
26399.6651599 27258.16594
27258.16594 27572.54024
27572.54024 27948.91412
27948.91412 28319.41471
28319.41471 28483.91471


In [62]:
arr[-1]

67.0

In [51]:
len(arr), len(summary)

(1559, 1562)

In [ ]:
def extend_array(array_to_extend, array_ref):
    '''
    This function takes two arrays, the first is the one to be extended and the second one the reference.
    It returns the new array which is 1D, being the time column skipped since it has been taken into accountin the processs. 
    '''
    
    array_size = np.size(array_ref, 0)
    new_array = []
    count = 0
    
    for i in range(array_size):

        if array_ref[i][0] > array_to_extend[count][0]:
            new_array.append(array_to_extend[count][1])
            count += 1
        else:
            new_array.append(-999)  # For the "missing values" it appends -999.
    
    return np.array(new_array)

In [ ]:
df = generate_dataset(os.path.join(motion_path, motion_list[0]), 
                 os.path.join(heart_rate_path, heart_rate_list[0]), 
                 os.path.join(labels_path, labels_list[0]))

1187904
4965
952


In [ ]:
# check Time: 25890.016651	27356.335354
df[df["Labels"] > -1].tail(50)

Time         X         Y         Z  Heart Rate  Labels
1150608  25530.003946  0.133011 -0.976974  0.162323      -999.0     2.0
1152107  25560.011476  0.132507 -0.975006  0.163269      -999.0     2.0
1153606  25590.017736  0.130524 -0.974991  0.165192      -999.0     2.0
1155104  25620.008041  0.133987 -0.975983  0.163788      -999.0     2.0
1156603  25650.013869  0.131546 -0.976486  0.161316      -999.0     2.0
1158101  25680.002878  0.133026 -0.976486  0.160858      -999.0     2.0
1159600  25710.008974  0.131531 -0.974518  0.163254      -999.0     2.0
1161099  25740.017955  0.132019 -0.974518  0.163254      -999.0     2.0
1162597  25770.002792  0.132980 -0.975479  0.165222      -999.0     2.0
1164096  25800.010575  0.130081 -0.977966  0.160812      -999.0     2.0
1167093  25860.019480  0.263855  1.263306  0.121704      -999.0     1.0
1168591  25890.016651  0.198242  0.927002  0.338501      -999.0     1.0
1178037  27356.335354 -0.036804  0.059677 -0.988281        60.0     1.0
1178038  27356.341649 -0.038269  0.057709 -0.989273        59.0     1.0
1178039  27356.376896 -0.037323  0.056747 -0.984863      -999.0     5.0
1178040  27356.377824 -0.038788  0.056732 -0.986832      -999.0     5.0
1178041  27356.398863 -0.037323  0.056747 -0.984375      -999.0     2.0
1178042  27356.401757 -0.035828  0.058212 -0.987289      -999.0     5.0
1178043  27356.403478 -0.036804  0.058701 -0.988281      -999.0     5.0
1178044  27356.404345 -0.037766  0.056229 -0.989731      -999.0     5.0
1178045  27356.408534 -0.035812  0.056229 -0.988739      -999.0     2.0
1178046  27356.409394 -0.038269  0.056732 -0.988769      -999.0     2.0
1178047  27356.410206 -0.038300  0.057724 -0.986832      -999.0     2.0
1178048  27356.411039 -0.037323  0.059204 -0.985367      -999.0     2.0
1178049  27356.411889 -0.036316  0.057724 -0.987289      -999.0     2.0
1178050  27356.412898 -0.037796  0.057236 -0.986343      -999.0     2.0
1178052  27356.414609 -0.038788  0.058212 -0.985870      -999.0     1.0
1178053  27356.415388 -0.037796  0.056244 -0.986328      -999.0     2.0
1178054  27356.417310 -0.036819  0.057724 -0.987305      -999.0     2.0
1178055  27356.424157 -0.035812  0.057221 -0.988739      -999.0     1.0
1178056  27356.425011 -0.037308  0.057236 -0.986816      -999.0     2.0
1178057  27356.427350 -0.037811  0.057724 -0.986343      -999.0     2.0
1178058  27356.428203 -0.036331  0.057724 -0.986313      -999.0     1.0
1178059  27356.428997 -0.035812  0.057221 -0.988251      -999.0     2.0
1178060  27356.430246 -0.037811  0.057236 -0.985855      -999.0     2.0
1178061  27356.431063 -0.036331  0.055756 -0.985809      -999.0     1.0
1178062  27356.431826 -0.037292  0.055756 -0.987289      -999.0     2.0
1178064  27356.433871 -0.037796  0.057236 -0.986832      -999.0     1.0
1178065  27356.434719 -0.037308  0.057724 -0.986328      -999.0     1.0
1178066  27356.435928 -0.036316  0.056732 -0.987289      -999.0     1.0
1178067  27356.447586 -0.036819  0.058701 -0.986816      -999.0     1.0
1178068  27356.448449 -0.036819  0.056747 -0.985825      -999.0     1.0
1178069  27356.449269 -0.036331  0.055756 -0.985809      -999.0     2.0
1178070  27356.450106 -0.035339  0.056732 -0.986786      -999.0     2.0
1178071  27356.450928 -0.038284  0.057221 -0.987320      -999.0     2.0
1178072  27356.451748 -0.037781  0.055252 -0.989243      -999.0     2.0
1178073  27356.452570 -0.037796  0.057724 -0.987320      -999.0     2.0
1178074  27356.453401 -0.039261  0.055252 -0.987808      -999.0     1.0
1178667  27360.008037  0.026413  0.134613 -0.981049      -999.0     2.0
1179117  27690.656066  0.024475  0.133133 -0.983505        69.0     2.0

In [ ]:
''' 
To do next:
    decision making: what to do with the missing inervals in the motion dataset
    divide within interval of time.
    
'''

' \nTo do next:\n    decision making: what to do with the missing inervals in the motion dataset\n    divide within interval of time.\n    \n'

###### Testing DataFrame

In [ ]:
# Cropping

(start, end) = crop_to_offset(user_1_heart_rate, user_1_labels)
user_1_heart_rate = user_1_heart_rate[start-1:end]

(start, end) = crop_to_offset(user_1_motion, user_1_labels)
user_1_motion = user_1_motion[start-1:end]

In [ ]:
# extending

array_size = np.size(user_1_motion, 0)
expanded_heart_rate = []
count_heart_rate = 0
expanded_labels = []
count_labels = 0

for i in range(array_size):
    
    # heart rate
    if user_1_motion[i][0] > user_1_heart_rate[count_heart_rate][0]:
        expanded_heart_rate.append(user_1_heart_rate[count_heart_rate][1])
        count_heart_rate += 1
    else:
        expanded_heart_rate.append(-999)
    
    # labels
    if user_1_motion[i][0] > user_1_labels[count_labels][0]:
        expanded_labels.append(user_1_labels[count_labels][1])
        count_labels += 1
    else:
        expanded_labels.append(-999)

Time         X         Y         Z  heart rate  labels
0           -0.004037  0.404434  0.446549 -0.796829        50.0    -1.0
1            0.015948  0.403931  0.449005 -0.796860        -1.0     0.0
2            0.036006  0.403915  0.448029 -0.795395        -1.0    -1.0
3            0.055885  0.404907  0.446549 -0.795853        -1.0    -1.0
4            0.075883  0.408356  0.447525 -0.796768        -1.0    -1.0
...               ...       ...       ...       ...         ...     ...
1187899  28394.149736 -0.601166 -0.075180 -0.774841        -1.0    -1.0
1187900  28394.169696 -0.600189 -0.071228 -0.774857        -1.0    -1.0
1187901  28394.189770 -0.599213 -0.069260 -0.773880        -1.0    -1.0
1187902  28394.209753 -0.597260 -0.072205 -0.771393        -1.0    -1.0
1187903  28394.229658 -0.596283 -0.074173 -0.770874         NaN     NaN

[1187904 rows x 6 columns]

In [ ]:
# creating dataframe and adding hr and lb to dataframe as columns

d = pd.DataFrame(user_1_motion, columns=['Time', 'X', 'Y', 'Z'])

hr = pd.Series(new_heart_rate)
d["Heart Rate"] = hr
lb = pd.Series(new_labels)
d["Labels"] = lb

d

In [ ]:
np.size(user_1_heart_rate, 0), np.size(user_1_labels, 0)

4965

In [ ]:
a = d[(d["Time"] > 25559) & (d["Time"] < 25591)]
a[a["labels"] > -1]

Time         X         Y         Z  heart rate  labels
1152107  25560.011476  0.132507 -0.975006  0.163269        -1.0     2.0
1153606  25590.017736  0.130524 -0.974991  0.165192        -1.0     2.0

## To do

- Get spike values for motion
- Get average 
- parameters variables, spike interval, interval epoch, choose types of median